In [1]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
project_dir = "/data/ephemeral/home/nlp-5/song/"
import os
import sys
sys.path.append(
    project_dir
)
from src.utils.utils import *

import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

train_df = pd.read_csv(os.path.join(project_dir, 'data','train.csv'))
val_df = pd.read_csv(os.path.join(project_dir, 'data','dev.csv'))

test_df = pd.read_csv(os.path.join(project_dir, 'data','test.csv'))
sub_df = pd.read_csv(os.path.join(project_dir, 'outputs', 'exp_aug_0730191312', 'submission_0730191312-0.csv'))

In [9]:
datasets = {
    'train': train_df,
    'dev': val_df,
    'test': test_df,
}

In [11]:
import re
from collections import Counter
import pandas as pd # pandas 라이브러리가 사용되므로 import 추가

### 1. 약어 / 이모티콘 빈도 분석
def extract_abbreviations(texts):
    """
    주어진 텍스트 목록에서 약어 또는 이모티콘으로 추정되는 패턴을 추출하고 빈도를 분석합니다.
    - 한글 자음/모음이 2개 이상 연속되거나 (ex: ㅋㅋ, ㅎㅎ, ㅠㅠ)
    - 영문 알파벳이 2개 이상 연속되는 (ex: lol, wtf) 패턴을 찾습니다.

    Args:
        texts (list or pd.Series): 분석할 텍스트 문자열들의 목록 또는 pandas Series.

    Returns:
        Counter: 추출된 약어/이모티콘과 그 빈도를 담은 Counter 객체.
    """
    # 정규 표현식 패턴 정의:
    # r'\b[ㄱ-ㅎㅏ-ㅣ]{2,}\b' : 단어 경계(\b) 내에서 한글 자음/모음이 2개 이상 반복 (예: ㅋㅋ, ㅠㅠ)
    # r'\b[a-zA-Z]{2,}\b'   : 단어 경계(\b) 내에서 영문 알파벳이 2개 이상 반복 (예: lol, omg)
    # r'[ㄱ-ㅎ]{2,}'         : 단어 경계에 상관없이 한글 자음이 2개 이상 반복 (예: ㅋㅋㅋ, ㅎㅎㅎ - 단어 중간에도 매칭)
    # 첫 번째와 세 번째 패턴은 한글 초성/중성을 주로 잡기 위함이고, 두 번째는 영문 약어를 위함입니다.
    pattern = r'\b[ㄱ-ㅎㅏ-ㅣ]{2,}\b|\b[a-zA-Z]{2,}\b|[ㄱ-ㅎ]{2,}'
    matches = [] # 추출된 약어/이모티콘을 저장할 리스트

    for text in texts:
        if pd.isna(text): # 텍스트가 NaN (결측값)인 경우 건너뛰기
            continue
        found = re.findall(pattern, text) # 현재 텍스트에서 패턴에 맞는 모든 문자열 찾기
        matches.extend(found) # 찾은 문자열들을 matches 리스트에 추가
    return Counter(matches) # matches 리스트의 각 항목 빈도를 계산하여 반환

print("\n [1] 약어/이모티콘 빈도 분석")
# 'datasets'는 외부에서 정의된 딕셔너리로 가정하며, 
# 각 키(name)는 데이터셋의 이름이고 값(df)은 pandas DataFrame입니다.
for name, df in datasets.items():
    count = extract_abbreviations(df['dialogue']) # 'dialogue' 열에서 약어/이모티콘 추출 및 빈도 계산
    print(f"\n{name.upper()} 상위 약어 100개:")
    print(count.most_common(100)) # 가장 빈번한 상위 20개 출력


### 2. 지시표현 비율
# 지시표현으로 간주할 단어들의 목록
deictic_words = ['그 사람', '이 사람', '그거', '이거', '그건', '이건', '거기', '저기', '여기']

def count_deictic_ratio(texts):
    """
    주어진 텍스트 목록에서 지시표현이 포함된 텍스트의 비율을 계산합니다.
    'deictic_words' 리스트에 정의된 단어 중 하나라도 텍스트에 포함되어 있으면 지시표현이 있는 것으로 간주합니다.

    Args:
        texts (list or pd.Series): 분석할 텍스트 문자열들의 목록 또는 pandas Series.

    Returns:
        float: 지시표현을 포함하는 텍스트의 비율 (소수점 넷째 자리까지 반올림).
               총 텍스트가 0개인 경우 0.0을 반환.
    """
    total, cnt = 0, 0 # total: 총 텍스트 수, cnt: 지시표현을 포함하는 텍스트 수
    for text in texts:
        if pd.isna(text): # 텍스트가 NaN (결측값)인 경우 건너뛰기
            continue
        total += 1 # 유효한 텍스트이므로 총 텍스트 수 증가
        # deictic_words 리스트의 어떤 단어라도 현재 텍스트에 포함되어 있는지 확인
        if any(word in text for word in deictic_words):
            cnt += 1 # 포함되어 있다면 지시표현 포함 텍스트 수 증가
    
    # 지시표현 포함 비율 계산 (총 텍스트 수가 0이 아니면 계산, 아니면 0.0 반환)
    return round(cnt / total, 4) if total > 0 else 0.0

print("\n [2] 지시표현 비율")
for name, df in datasets.items():
    ratio = count_deictic_ratio(df['dialogue']) # 'dialogue' 열에서 지시표현 포함 비율 계산
    print(f"{name.upper()} 지시표현 포함 비율: {ratio*100:.2f}%") # 백분율로 출력


### 3. 요약문 길이 분석 (train/dev만)
def get_length_stats(series):
    """
    주어진 텍스트 시리즈(주로 요약문)의 길이에 대한 통계(개수, 최소, 최대, 평균, 중앙값)를 계산합니다.
    텍스트 길이는 공백으로 구분된 단어의 수로 정의됩니다.

    Args:
        series (pd.Series): 길이를 분석할 문자열(요약문)이 담긴 pandas Series.

    Returns:
        dict: 'count', 'min', 'max', 'mean', 'median' 키를 가진 통계 결과 딕셔너리.
    """
    # 결측값을 제거하고 문자열 타입으로 변환 후, 각 문자열을 공백으로 분리하여 단어 수(길이) 계산
    lengths = series.dropna().astype(str).apply(lambda x: len(x.split()))
    return {
        'count': len(lengths), # 유효한 요약문의 개수
        'min': lengths.min(),  # 최소 길이
        'max': lengths.max(),  # 최대 길이
        'mean': round(lengths.mean(), 2), # 평균 길이 (소수점 둘째 자리까지 반올림)
        'median': lengths.median() # 중앙값 길이
    }

print("\n [3] 요약문 단어 개수 분석 (train/dev)")
# 'train'과 'dev' 데이터셋에 대해서만 요약문 길이 분석 수행
for name in ['train', 'dev']:
    # datasets 딕셔너리에서 해당 데이터셋의 'summary' 열을 가져와 통계 계산
    stats = get_length_stats(datasets[name]['summary'])
    print(f"\n{name.upper()} 요약 길이 통계:")
    for k, v in stats.items():
        print(f"{k}: {v}")


### 4. 중복 요약 확인 (train/dev만)
print("\n [4] 중복 요약 비율 (train/dev)")
# 'train'과 'dev' 데이터셋에 대해서만 중복 요약 확인 수행
for name in ['train', 'dev']:
    # 'summary' 열에서 결측값을 제거한 요약문들 가져오기
    summaries = datasets[name]['summary'].dropna()
    total = len(summaries) # 총 요약문의 개수
    duplicated = summaries.duplicated().sum() # 중복되는 요약문의 개수 계산
    
    # 중복 비율 계산 (총 요약문이 0개가 아니면 계산, 아니면 0.0 반환)
    ratio = duplicated / total if total > 0 else 0.0
    print(f"{name.upper()}")


 [1] 약어/이모티콘 빈도 분석

TRAIN 상위 약어 100개:
[('Mr', 748), ('TV', 230), ('PhoneNumber', 228), ('Mary', 190), ('Smith', 149), ('Mrs', 136), ('Tom', 134), ('John', 121), ('Bob', 101), ('Dr', 94), ('Jack', 92), ('Miss', 91), ('Address', 89), ('Jane', 84), ('Mike', 81), ('David', 76), ('Ms', 76), ('Brown', 75), ('Bill', 66), ('Susan', 63), ('Jim', 62), ('Steven', 62), ('Green', 58), ('Li', 58), ('White', 55), ('Wang', 49), ('Sam', 44), ('Alice', 42), ('Mark', 42), ('Paul', 41), ('CD', 40), ('Jenny', 39), ('The', 39), ('Peter', 38), ('James', 37), ('Ann', 37), ('Tim', 35), ('Sarah', 35), ('ABC', 34), ('Johnson', 34), ('of', 33), ('Helen', 32), ('Jones', 32), ('Kate', 31), ('Taylor', 31), ('Janet', 30), ('George', 30), ('Amy', 29), ('the', 28), ('Daniel', 27), ('Lily', 27), ('Steve', 27), ('Sally', 27), ('Black', 27), ('Jason', 26), ('Michael', 26), ('Lucy', 25), ('IT', 25), ('Anna', 25), ('Betty', 25), ('Liu', 25), ('DVD', 24), ('Nancy', 24), ('Rick', 24), ('Karen', 24), ('DateOfBirth', 24), ('Li

In [14]:
### 1. 약어 / 이모티콘 빈도 분석
def extract_abbreviations(texts):
    """
    주어진 텍스트 목록에서 약어 또는 이모티콘으로 추정되는 패턴을 추출하고 빈도를 분석합니다.
    - 한글 자음/모음이 2개 이상 연속되거나 (ex: ㅋㅋ, ㅎㅎ, ㅠㅠ)
    - 영문 알파벳이 2개 이상 연속되는 (ex: lol, wtf) 패턴을 찾습니다.

    Args:
        texts (list or pd.Series): 분석할 텍스트 문자열들의 목록 또는 pandas Series.

    Returns:
        Counter: 추출된 약어/이모티콘과 그 빈도를 담은 Counter 객체.
    """
    # 정규 표현식 패턴 정의:
    # r'\b[ㄱ-ㅎㅏ-ㅣ]{2,}\b' : 단어 경계(\b) 내에서 한글 자음/모음이 2개 이상 반복 (예: ㅋㅋ, ㅠㅠ)
    # r'\b[a-zA-Z]{2,}\b'   : 단어 경계(\b) 내에서 영문 알파벳이 2개 이상 반복 (예: lol, omg)
    # r'[ㄱ-ㅎ]{2,}'         : 단어 경계에 상관없이 한글 자음이 2개 이상 반복 (예: ㅋㅋㅋ, ㅎㅎㅎ - 단어 중간에도 매칭)
    # 첫 번째와 세 번째 패턴은 한글 초성/중성을 주로 잡기 위함이고, 두 번째는 영문 약어를 위함입니다.
    pattern = r'[ㄱ-ㅎ]{2,}'
    matches = [] # 추출된 약어/이모티콘을 저장할 리스트

    for text in texts:
        if pd.isna(text): # 텍스트가 NaN (결측값)인 경우 건너뛰기
            continue
        found = re.findall(pattern, text) # 현재 텍스트에서 패턴에 맞는 모든 문자열 찾기
        matches.extend(found) # 찾은 문자열들을 matches 리스트에 추가
    return Counter(matches) # matches 리스트의 각 항목 빈도를 계산하여 반환

print("\n [1] 약어/이모티콘 빈도 분석")
# 'datasets'는 외부에서 정의된 딕셔너리로 가정하며, 
# 각 키(name)는 데이터셋의 이름이고 값(df)은 pandas DataFrame입니다.
for name, df in datasets.items():
    count = extract_abbreviations(df['dialogue']) # 'dialogue' 열에서 약어/이모티콘 추출 및 빈도 계산
    print(f"\n{name.upper()} 상위 약어 100개:")
    print(count.most_common(100)) # 가장 빈번한 상위 20개 출력


 [1] 약어/이모티콘 빈도 분석

TRAIN 상위 약어 100개:
[('ㅎㅎ', 1)]

DEV 상위 약어 100개:
[]

TEST 상위 약어 100개:
[]


In [38]:
print(train_df[train_df['dialogue'].str.contains('<br>')]['dialogue'].values[0])
print(val_df[val_df['dialogue'].str.contains('<br>')]['dialogue'].values)

#Person1#: 요즘 잘 지내고 있어요?<br>#Person2#: 제 코치가 제 혈압을 체크해 달라고 부탁했어요.<br>#Person1#: 전에 고혈압 있다고 들은 적 있나요?<br>#Person2#: 고혈압 증상은 없어요.<br>#Person1#: 고혈압 있는 사람들은 대부분 본인이 모르는 경우가 많아요. 그래서 조용한 살인자라고 부르죠.<br>#Person2#: 고혈압 검사는 어떻게 해요?<br>#Person1#: 이 커프에 공기를 넣어서 측정할 거예요.<br>#Person2#: 측정 후에 어떤 정보를 알 수 있나요?<br>#Person1#: 심장이 얼마나 강하게 피를 펌프하는지와 동맥이 쉴 때 얼마나 이완되는지를 알려줘요.<br>#Person2#: 이번 검사가 잘 됐으면 좋겠어요.
[]


In [21]:
print(train_df[train_df['dialogue'].str.contains('ㅎㅎ')]['summary'].values[0])

#Person1#과 #Person2#는 전 남자친구에 대해 이야기한 후, #Person1#이 #Person2#에게 사랑을 고백하고, #Person2#는 #Person1#의 여자친구가 되기로 합니다. 둘은 서로의 감정을 나누며 행복해합니다.


In [40]:
text = "<br> sdfld <br> dsfsdf"
text = text.replace('<br>', '\\n')
print(text)

\n sdfld \n dsfsdf
